In [2]:
import numpy as np
import pandas as pd
import math

In [7]:
def monthly_return(yearly_return:float):
  """
  Calculate the monthly return of an excepted yearly return.
  """
  return pow(1 + yearly_return, 1/12) - 1

In [18]:
def automatic_investment(monthly_return: float, monthly_investment: float, period: int):
  """
  Calculate the balance after a certain period of time with a certain monthly investment and monthly return.
  """
  month_num = period * 12
  balance = monthly_investment

  for i in range(month_num):
    balance = balance * (1 + monthly_return) + monthly_investment

  return balance

In [31]:
yearly_return = 0.1
investment_period = 20
monthly_investment = 10

In [32]:
print(f"经过20年的总余额是： {automatic_investment(monthly_return(yearly_return), monthly_investment, investment_period)} \n \
共盈利 {automatic_investment(monthly_return(yearly_return), monthly_investment, investment_period) - investment_period*12*monthly_investment}")

经过20年的总余额是： 7249.867280618935 
 共盈利 4849.867280618935
